In [ ]:
import numpy
import torch

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import src.point_data

---

## Problem Setup

In [ ]:
plt.figure(figsize=(10, 10))

target_slope_val = 3.5
xy = src.point_data.generate_center_line_points(
    m_mu=target_slope_val,
    m_std=1.0,
    num_pts=100
)

plt.scatter(xy[:, 0], xy[:, 1])
plt.grid()

plt.axhline(0, color="black")
plt.axvline(0, color="black")

plt.axline((0, 0), slope=target_slope_val, color="gray", linestyle="--")

plt.show()

In [ ]:
slope_param = torch.FloatTensor([0])

In [ ]:
slope_param

In [ ]:
def compute_loss(slope_param, xy):
    return torch.sum(torch.square(xy[:, 1] - (slope_param * xy[:, 0])))

In [ ]:
slope_val_arr = numpy.linspace(-100, 100, 1000)
slope_val_loss_list = []

for slope_val in slope_val_arr:
    slope_param[0] = slope_val
    slope_val_loss_list.append(compute_loss(slope_param, xy))

In [ ]:
plt.figure(figsize=(10, 10))

plt.plot(slope_val_arr, slope_val_loss_list)

plt.xlabel("Line slope")
plt.ylabel("Line loss")

plt.axvline(target_slope_val, color="red")

plt.grid()
plt.show()

---

## Parameters & Optimizer

In [ ]:
slope_param = torch.nn.Parameter(torch.FloatTensor([0]), requires_grad=True)

In [ ]:
optimizer = torch.optim.SGD(
    params=(slope_param,),
    lr=1e-2
)

In [ ]:
slope_param

In [ ]:
slope_param.grad

In [ ]:
loss = compute_loss(slope_param, xy)

In [ ]:
loss

In [ ]:
loss.backward()

In [ ]:
slope_param

In [ ]:
slope_param.grad

In [ ]:
optimizer.step()

In [ ]:
slope_param

In [ ]:
slope_param.grad

In [ ]:
optimizer.zero_grad()

In [ ]:
slope_param

In [ ]:
slope_param.grad

---

## First Training Loop

In [ ]:
slope_param = torch.nn.Parameter(torch.FloatTensor([0]), requires_grad=True)

# 0.02
# 0.03
# 0.035
# 0.0325

optimizer = torch.optim.SGD(
    params=(slope_param,),
    lr=0.0325
)

#

training_slope_list = []
training_loss_list = []

num_epochs = 20

for epoch_idx in range(num_epochs):

    loss = compute_loss(slope_param, xy)
    loss.backward()

    training_slope_list.append(slope_param.item())
    training_loss_list.append(loss.item())

    optimizer.step()
    optimizer.zero_grad()

#

loss = compute_loss(slope_param, xy)

training_slope_list.append(slope_param.item())
training_loss_list.append(loss.item())

In [ ]:
plt.figure(figsize=(15, 15))

plt.plot(slope_val_arr, slope_val_loss_list)

plt.plot(
    training_slope_list,
    training_loss_list,
    color="black",
    alpha=0.5,
    label="Loss Curve"
)

cmap_name = "plasma"

scatter = plt.scatter(
    training_slope_list,
    training_loss_list,
    color=numpy.asarray([
        mpl.colormaps.get_cmap(cmap_name)(x)
        for x in numpy.linspace(0, 1, len(training_slope_list))
    ]),
    edgecolor="black",
    s=100,
    label="Training Epochs"
)

plt.xlabel("Line slope")
plt.ylabel("Line loss")

plt.axvline(target_slope_val, color="red")

sm = plt.cm.ScalarMappable(cmap=mpl.colormaps.get_cmap(cmap_name))
sm.set_clim(vmin=0, vmax=num_epochs)

plt.colorbar(
    sm,
    ax=plt.gca()
)

plt.grid()
plt.legend()

#plt.xlim(-5, 15)
#plt.ylim(0, 500)

plt.show()

In [ ]:
print("Final loss: {:.2e}".format(training_loss_list[-1]))

In [ ]:
plt.figure(figsize=(10, 10))

plt.scatter(xy[:, 0], xy[:, 1])
plt.grid()

plt.axhline(0, color="black")
plt.axvline(0, color="black")

plt.axline((0, 0), slope=slope_param.item(), color="red")
plt.axline((0, 0), slope=target_slope_val, color="gray", linestyle="--")

plt.show()

---

## Using a Learning Rate Scheduler

In [ ]:
slope_param = torch.nn.Parameter(torch.FloatTensor([0]), requires_grad=True)

# 0.02
# 0.03
# 0.05

optimizer = torch.optim.SGD(
    params=(slope_param,),
    lr=0.05
)

scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer=optimizer,
    gamma=0.9
)

#

training_slope_list = []
training_lr_list = []
training_loss_list = []

num_epochs = 20

for epoch_idx in range(num_epochs):
    
    loss = compute_loss(slope_param, xy)
    loss.backward()

    training_slope_list.append(slope_param.item())
    training_loss_list.append(loss.item())

    optimizer.step()
    optimizer.zero_grad()

    training_lr_list.append(scheduler.get_last_lr())

    scheduler.step()

#

loss = compute_loss(slope_param, xy)

training_slope_list.append(slope_param.item())
training_loss_list.append(loss.item())

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(training_lr_list, "-o")

plt.xlabel("Epoch")
plt.ylabel("Learning Rate")

plt.grid()

plt.show()

In [ ]:
plt.figure(figsize=(15, 15))

plt.plot(slope_val_arr, slope_val_loss_list)

plt.plot(
    training_slope_list,
    training_loss_list,
    color="black",
    alpha=0.5,
    label="Loss Curve"
)

cmap_name = "plasma"

scatter = plt.scatter(
    training_slope_list,
    training_loss_list,
    color=numpy.asarray([
        mpl.colormaps.get_cmap(cmap_name)(x)
        for x in numpy.linspace(0, 1, len(training_slope_list))
    ]),
    edgecolor="black",
    s=100,
    label="Training Epochs"
)

plt.xlabel("Line slope")
plt.ylabel("Line loss")

plt.axvline(target_slope_val, color="red")

sm = plt.cm.ScalarMappable(cmap=mpl.colormaps.get_cmap(cmap_name))
sm.set_clim(vmin=0, vmax=num_epochs)

plt.colorbar(
    sm,
    ax=plt.gca()
)

plt.grid()
plt.legend()

#plt.xlim(-5, 15)
#plt.ylim(0, 500)

plt.show()

In [ ]:
print("Final loss: {:.2e}".format(training_loss_list[-1]))

In [ ]:
plt.figure(figsize=(10, 10))

plt.scatter(xy[:, 0], xy[:, 1])
plt.grid()

plt.axhline(0, color="black")
plt.axvline(0, color="black")

plt.axline((0, 0), slope=slope_param.item(), color="red")
plt.axline((0, 0), slope=target_slope_val, color="gray", linestyle="--")

plt.show()

---

## Training Loop with Batches

In [ ]:
xy.shape

In [ ]:
xy_batch_list = torch.split(xy, 10, dim=0)

print(len(xy_batch_list))
for xy_batch in xy_batch_list:
    print(xy_batch.shape)

In [ ]:
slope_param = torch.nn.Parameter(torch.FloatTensor([0]), requires_grad=True)

optimizer = torch.optim.SGD(
    params=(slope_param,),
    lr=0.035 * 10
)

scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer=optimizer,
    gamma=0.9
)

#

training_slope_list = []
training_lr_list = []
training_loss_list = []

num_epochs = 20
batch_size = 10

for epoch_idx in range(num_epochs):

    with torch.no_grad():

        epoch_total_loss = 0

        for xy_batch in torch.split(xy, batch_size, dim=0):
            loss = compute_loss(slope_param, xy_batch)
            epoch_total_loss += loss.item()
        
    training_slope_list.append(slope_param.item())
    training_loss_list.append(epoch_total_loss)

    #

    for xy_batch in torch.split(xy, batch_size, dim=0):

        loss = compute_loss(slope_param, xy_batch)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()

    training_lr_list.append(scheduler.get_last_lr())

    scheduler.step()

#

final_loss = 0

with torch.no_grad():
    for xy_batch in torch.split(xy, batch_size, dim=0):
        loss = compute_loss(slope_param, xy_batch)
        final_loss += loss.item()

training_slope_list.append(slope_param.item())
training_loss_list.append(final_loss)

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(training_lr_list, "-o")

plt.xlabel("Epoch")
plt.ylabel("Learning Rate")

plt.grid()

plt.show()

In [ ]:
plt.figure(figsize=(15, 15))

plt.plot(slope_val_arr, slope_val_loss_list)

plt.plot(
    training_slope_list,
    training_loss_list,
    color="black",
    alpha=0.5,
    label="Loss Curve"
)

cmap_name = "plasma"

scatter = plt.scatter(
    training_slope_list,
    training_loss_list,
    color=numpy.asarray([
        mpl.colormaps.get_cmap(cmap_name)(x)
        for x in numpy.linspace(0, 1, len(training_slope_list))
    ]),
    edgecolor="black",
    s=100,
    label="Training Epochs"
)

plt.xlabel("Line slope")
plt.ylabel("Line loss")

plt.axvline(target_slope_val, color="red")

sm = plt.cm.ScalarMappable(cmap=mpl.colormaps.get_cmap(cmap_name))
sm.set_clim(vmin=0, vmax=num_epochs)

plt.colorbar(
    sm,
    ax=plt.gca()
)

plt.grid()
plt.legend()

#plt.xlim(-5, 15)
#plt.ylim(0, 500)

plt.show()

In [ ]:
print("Final loss: {:.2e}".format(training_loss_list[-1]))

In [ ]:
plt.figure(figsize=(10, 10))

plt.scatter(xy[:, 0], xy[:, 1])
plt.grid()

plt.axhline(0, color="black")
plt.axvline(0, color="black")

plt.axline((0, 0), slope=slope_param.item(), color="red")
plt.axline((0, 0), slope=target_slope_val, color="gray", linestyle="--")

plt.show()